# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
print(tf.__version__)

2.18.0


### Preprocessing the Training set

In [3]:
# Create the dataset
train_set = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    image_size=(64, 64),
    batch_size=32,
    label_mode='binary'
)

Found 1498 files belonging to 2 classes.


In [4]:
# Data augmentation layer
train_data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),  # Rescale pixel values
    tf.keras.layers.RandomFlip('horizontal'),  # Apply horizontal flip
    tf.keras.layers.RandomZoom(0.2),  # Apply zoom augmentation
    tf.keras.layers.RandomRotation(0.2),  # Apply rotation for better variety
    tf.keras.layers.RandomShear(0.2)  # Apply shear transformation
])

In [5]:
# Apply the augmentation pipeline
augmented_train_dataset = train_set.map(lambda x, y: (train_data_augmentation(x, training=True), y))

### Preprocessing the Test set

In [6]:
test_set = tf.keras.utils.image_dataset_from_directory(
    'dataset/test',
    image_size=(64, 64),
    batch_size=32,
    label_mode='binary',
    shuffle=False
)

Found 358 files belonging to 2 classes.


In [7]:
test_data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255)
])

In [8]:
augmented_test_dataset = test_set.map(lambda x, y: (test_data_augmentation(x, training=False), y))

### Initialising the CNN

In [9]:
cnn = tf.keras.models.Sequential()


### Step 1 - Convolution

In [10]:
cnn.add(tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(64, 64, 3)))

C:\Users\lenovo\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [11]:
cnn.add(tf.keras.layers.MaxPooling2D(2))


### Adding a second convolutional layer

In [12]:
cnn.add(tf.keras.layers.Conv2D(64, 3, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(2))

### Step 3 - Flattening

In [13]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [14]:
cnn.add(tf.keras.layers.Dense(128, activation='relu'))

In [15]:
cnn.add(tf.keras.layers.Dropout(0.3))

### Step 5 - Output Layer

In [16]:
cnn.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # For binary classification

### Compiling the CNN

In [17]:
cnn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)



### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(
    x=augmented_train_dataset,
    validation_data=augmented_test_dataset,
    epochs=25
)


Epoch 1/25
45/47 ━━━━━━━━━━━━━━━━━━━━ 1s 866ms/step - accuracy: 0.5410 - loss: 0.7035

## Part 4 - Making a single prediction

In [ ]:
test_image = tf.keras.utils.load_img(
   'single_prediction/sample.jpg',
    target_size=(64, 64),
)
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


In [ ]:
print("Prediction Score:", result[0][0])

if result[0][0] > 0.5:
    print("Predicted Class: Defective Tyre")
else:
    print("Predicted Class: Good Tyre")


Prediction Score: 1.0049782e-17
Predicted Class: Good Tyre
